In [ ]:
# Boto3 SageMaker Invoke Endpoint
# This example shows how to invoke SageMaker Endpoint from outside of AWS environment using Boto3 SDK
# Boto is the Amazon Web Services (AWS) SDK for Python
# https://boto3.amazonaws.com/v1/documentation/api/latest/index.html

# Endpoint: XGBoost - Kaggle Bike Rental - Regressor Trained in XGBoost Lectures
# Makesure Endpoint is deployed before running this example
# 
# Reference:
#  https://github.com/awslabs/amazon-sagemaker-examples

# NOTE: SageMaker SDK now requires additional permissions DescribeEndpoint, DescribeEndpointConfig in-addition to InvokeEndpoint
#   boto3 SDK requires just InvokeEndpoint permission.
#   Please update SageMakerInvokeEndpoint permissions to reflect this policy document:
#   Logon with my_admin account and update permissions (IAM->Policies->SageMakerInvokeEndpoint->Edit Policy)
#   
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "sagemaker:DescribeEndpointConfig",
                "sagemaker:DescribeEndpoint",
                "sagemaker:InvokeEndpoint"
            ],
            "Resource": "*"
        }
    ]
}

In [1]:
import boto3
import math
import dateutil

In [2]:
# Establish a session with AWS
# Specify credentials and region to be used for this session.
# We will use a ml_user_predict credentials that has limited privileges
boto_session = boto3.Session(profile_name='ml_user',region_name='us-east-1')

In [3]:
# List of low level clients that are available in boto3
print(boto_session.get_available_services())

['accessanalyzer', 'acm', 'acm-pca', 'alexaforbusiness', 'amp', 'amplify', 'amplifybackend', 'apigateway', 'apigatewaymanagementapi', 'apigatewayv2', 'appconfig', 'appflow', 'appintegrations', 'application-autoscaling', 'application-insights', 'applicationcostprofiler', 'appmesh', 'apprunner', 'appstream', 'appsync', 'athena', 'auditmanager', 'autoscaling', 'autoscaling-plans', 'backup', 'batch', 'braket', 'budgets', 'ce', 'chime', 'chime-sdk-identity', 'chime-sdk-messaging', 'cloud9', 'clouddirectory', 'cloudformation', 'cloudfront', 'cloudhsm', 'cloudhsmv2', 'cloudsearch', 'cloudsearchdomain', 'cloudtrail', 'cloudwatch', 'codeartifact', 'codebuild', 'codecommit', 'codedeploy', 'codeguru-reviewer', 'codeguruprofiler', 'codepipeline', 'codestar', 'codestar-connections', 'codestar-notifications', 'cognito-identity', 'cognito-idp', 'cognito-sync', 'comprehend', 'comprehendmedical', 'compute-optimizer', 'config', 'connect', 'connect-contact-lens', 'connectparticipant', 'cur', 'customer-pr

In [4]:
# Acquire a SageMaker Runtime client for us-east-1 region
client = boto_session.client(service_name='sagemaker-runtime',region_name='us-east-1')

In [5]:
# Specify Your Endpoint Name
endpoint_name = 'xgboost-bikerental-v1'

In [6]:
# Raw Data
#datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
# Actual=562
sample_one = '2012-12-19 17:00:00,4,0,1,1,16.4,20.455,50,26.0027'
# Actual=569
sample_two = '2012-12-19 18:00:00,4,0,1,1,15.58,19.695,50,23.9994'
# Actual=4
sample_three = '2012-12-10 01:00:00,4,0,1,2,14.76,18.94,100,0'

In [7]:
# Raw Data Structure: 
# datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count

# Model expects data in this format (it was trained with these features):
# season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour

def transform_data(data):
    features = data.split(',')
    
    # Extract year, month, day, dayofweek, hour
    dt = dateutil.parser.parse(features[0])

    features.append(str(dt.year))
    features.append(str(dt.month))
    features.append(str(dt.day))
    features.append(str(dt.weekday()))
    features.append(str(dt.hour))
    
    # Return the transformed data. skip datetime field
    return ','.join(features[1:])

In [8]:
print('Raw Data:\n',sample_one)
print('Transformed Data:\n',transform_data(sample_one))

Raw Data:
 2012-12-19 17:00:00,4,0,1,1,16.4,20.455,50,26.0027
Transformed Data:
 4,0,1,1,16.4,20.455,50,26.0027,2012,12,19,2,17


In [9]:
# Let's invoke prediction now
result = client.invoke_endpoint(EndpointName=endpoint_name, 
                       Body=transform_data(sample_one).encode('utf-8'),
                       ContentType='text/csv')

In [10]:
result

{'ResponseMetadata': {'RequestId': 'cc9383c3-32b3-4bc1-a114-70202bbf2c22',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cc9383c3-32b3-4bc1-a114-70202bbf2c22',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Fri, 03 Sep 2021 05:39:42 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '17'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x22a20ab1b50>}

In [11]:
result = result['Body'].read().decode('utf-8')

In [12]:
print(result)

573.6282958984375


In [ ]:
# Actual Count is 562...but predicted is 6.36.

# Model was trained with log1p(count)
# So, we need to apply inverse transformation to get the actual count
# Predicted Count looks much better now
# print ('Predicted Count', math.expm1(float(result)))

In [13]:
print('\n'.join([transform_data(sample_one),transform_data(sample_two)]))

4,0,1,1,16.4,20.455,50,26.0027,2012,12,19,2,17
4,0,1,1,15.58,19.695,50,23.9994,2012,12,19,2,18


In [14]:
# Prediction for multiple observations in the same call
result = client.invoke_endpoint(EndpointName=endpoint_name, 
                       Body=('\n'.join([transform_data(sample_one),
                                        transform_data(sample_two)]).encode('utf-8')),
                       ContentType='text/csv')

In [15]:
result = result['Body'].read().decode('utf-8')

In [16]:
result

'573.6282958984375,547.5216064453125'

In [24]:
# Batch Prediction
# Transform data and invoke prediction in specified batch sizes
def run_predictions(data, batch_size):
    
    predictions = []
    
    transformed_data = [transform_data(row.strip()) for row in data]
    
    for i in range(0, len(data), batch_size):
        
        print(i,i+batch_size)
        
        result = client.invoke_endpoint(EndpointName=endpoint_name, 
                       Body=('\n'.join(transformed_data[i : i + batch_size]).encode('utf-8')),
                       ContentType='text/csv')
        
        result = result['Body'].read().decode('utf-8')
        result = result.split(',')
        #predictions += result
        predictions += [(float(r)) for r in result]
                
    return predictions

In [25]:
run_predictions([sample_one,sample_two,sample_three],10)

0 10


[573.6282958984375, 547.5216064453125, 10.423816680908203]

In [26]:
# Run a batch prediction on Test.CSV File
# Read the file content
data = []
with open('test.csv','r') as f:
    # skip header
    f.readline()
    # Read remaining lines
    data = f.readlines()

In [27]:
len(data)

6493

In [28]:
%%time
predictions = run_predictions(data,10)

0 10
10 20
20 30
30 40
40 50
50 60
60 70
70 80
80 90
90 100
100 110
110 120
120 130
130 140
140 150
150 160
160 170
170 180
180 190
190 200
200 210
210 220
220 230
230 240
240 250
250 260
260 270
270 280
280 290
290 300
300 310
310 320
320 330
330 340
340 350
350 360
360 370
370 380
380 390
390 400
400 410
410 420
420 430
430 440
440 450
450 460
460 470
470 480
480 490
490 500
500 510
510 520
520 530
530 540
540 550
550 560
560 570
570 580
580 590
590 600
600 610
610 620
620 630
630 640
640 650
650 660
660 670
670 680
680 690
690 700
700 710
710 720
720 730
730 740
740 750
750 760
760 770
770 780
780 790
790 800
800 810
810 820
820 830
830 840
840 850
850 860
860 870
870 880
880 890
890 900
900 910
910 920
920 930
930 940
940 950
950 960
960 970
970 980
980 990
990 1000
1000 1010
1010 1020
1020 1030
1030 1040
1040 1050
1050 1060
1060 1070
1070 1080
1080 1090
1090 1100
1100 1110
1110 1120
1120 1130
1130 1140
1140 1150
1150 1160
1160 1170
1170 1180
1180 1190
1190 1200
1200 1210
1210 1220

In [29]:
len(predictions),len(data)

(6493, 6493)

In [30]:
run_predictions(data,10)

0 10
10 20
20 30
30 40
40 50
50 60
60 70
70 80
80 90
90 100
100 110
110 120
120 130
130 140
140 150
150 160
160 170
170 180
180 190
190 200
200 210
210 220
220 230
230 240
240 250
250 260
260 270
270 280
280 290
290 300
300 310
310 320
320 330
330 340
340 350
350 360
360 370
370 380
380 390
390 400
400 410
410 420
420 430
430 440
440 450
450 460
460 470
470 480
480 490
490 500
500 510
510 520
520 530
530 540
540 550
550 560
560 570
570 580
580 590
590 600
600 610
610 620
620 630
630 640
640 650
650 660
660 670
670 680
680 690
690 700
700 710
710 720
720 730
730 740
740 750
750 760
760 770
770 780
780 790
790 800
800 810
810 820
820 830
830 840
840 850
850 860
860 870
870 880
880 890
890 900
900 910
910 920
920 930
930 940
940 950
950 960
960 970
970 980
980 990
990 1000
1000 1010
1010 1020
1020 1030
1030 1040
1040 1050
1050 1060
1060 1070
1070 1080
1080 1090
1090 1100
1100 1110
1110 1120
1120 1130
1130 1140
1140 1150
1150 1160
1160 1170
1170 1180
1180 1190
1190 1200
1200 1210
1210 1220

[10.033907890319824,
 -1.0149911642074585,
 -5.805891036987305,
 -4.739463806152344,
 -4.739463806152344,
 8.249551773071289,
 36.08793640136719,
 120.37787628173828,
 241.7802734375,
 122.58042907714844,
 25.992021560668945,
 24.739145278930664,
 60.00148391723633,
 54.010765075683594,
 53.65636444091797,
 58.378822326660156,
 88.44225311279297,
 181.41421508789062,
 188.09225463867188,
 121.68423461914062,
 76.15986633300781,
 50.02860641479492,
 34.80950927734375,
 27.765661239624023,
 15.826193809509277,
 -0.19815224409103394,
 -1.9352824687957764,
 -5.764981269836426,
 -3.4066741466522217,
 4.404345512390137,
 32.787906646728516,
 92.38922119140625,
 209.5614776611328,
 99.33727264404297,
 42.45364761352539,
 44.90315246582031,
 49.014747619628906,
 43.53658676147461,
 50.86595153808594,
 57.75297164916992,
 88.85710906982422,
 179.00228881835938,
 141.1117401123047,
 74.53825378417969,
 39.761749267578125,
 28.257423400878906,
 30.802831649780273,
 24.77512550354004,
 31.80458831